<a href="https://colab.research.google.com/github/HaiwenGuan/MLP-MNIST/blob/main/MLP_fmnist_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import numpy as np
import time
import tensorflow as tf
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from google.colab import drive
import math
#drive.mount('/content/drive')

np.random.seed(4208)
tf.random.set_seed(4208)

tf.executing_eagerly()
tf.__version__

tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
size_input = 784
size_hidden_1 = 256
size_hidden_2 = 128
size_output = 10

In [5]:
np.random.seed(4208)
tf.random.set_seed(4208)

In [6]:
(X_train, y_train),(X_test, y_test) = fashion_mnist.load_data()
X_train = tf.cast(tf.reshape(X_train, (-1, 784)), dtype=tf.float32)
X_test = tf.cast(tf.reshape(X_test, (-1, 784)), dtype=tf.float32)
X_train, X_test = X_train / 255.0, X_test / 255.0
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(100)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(20)

4431872/4422102 [==============================] - 0s 0us/step


In [13]:
class MLP(object):
  def __init__(self, size_input, size_hidden_1, size_hidden_2, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: etiher 'cpu' or 'gpu' or None. If None, decided automatically during eager.
    """
    self.size_input, self.size_hidden_1, self.size_hidden_2, self.size_output, self.device =\
    size_input, size_hidden_1, size_hidden_2, size_output, device

    initializer = tf.keras.initializers.HeUniform()

    #Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(initializer(shape=(self.size_input,self.size_hidden_1)))
    self.b1 = tf.Variable(initializer(shape=(1,self.size_hidden_1)))
    self.W2 = tf.Variable(initializer(shape=(self.size_hidden_1, self.size_hidden_2)))
    self.b2 = tf.Variable(initializer(shape=(1,self.size_hidden_2)))
    self.Wop = tf.Variable(initializer(shape=(self.size_hidden_2, self.size_output)))
    self.bop = tf.Variable(initializer(shape=(1,self.size_output)))

    self.variables = [self.W1, self.W2, self.Wop, self.b1, self.b2, self.bop]  # define variables(parameters) that will be updated

  def forward(self, X): # (X is the input matrix)
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
    
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape(batch_size, size_output)
    y_true - Tensor of shape(batch_size, size_output)
    '''
    #y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return scce(y_true, y_pred)

  def loss_l1(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape(batch_size, size_output)
    y_true - Tensor of shape(batch_size, size_output)
    '''
    #y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    penalty = (tf.reduce_sum(tf.math.abs(self.W1)).numpy() + tf.reduce_sum(tf.math.abs(self.W2)).numpy()\
               + tf.reduce_sum(tf.math.abs(self.Wop).numpy())) \
               / ( y_pred.shape[1] )
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return scce(y_true, y_pred) + penalty

  def loss_l2(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape(batch_size, size_output)
    y_true - Tensor of shape(batch_size, size_output)
    '''
    #y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    penalty = (tf.reduce_sum(tf.math.square(self.W1)).numpy() + tf.reduce_sum(tf.math.square(self.W2)).numpy() \
               + tf.reduce_sum(tf.math.square(self.Wop)).numpy())\
               / ( y_pred.shape[1] )
    #penalty = self.l2 * (tf.reduce_sum(tf.math.square(self.W1)).numpy() + tf.reduce_sum(tf.math.square(self.W2)).numpy() + tf.reduce_sum(tf.math.square(self.Wop).numpy()))
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return scce(y_true, y_pred) + penalty

  def backward(self, X_train, y_train):
    '''
    backward pass
    '''
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    with tf.GradientTape() as tape:
      tape.watch(self.variables)
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients( zip( grads, self.variables))

  def backward_1(self, X_train, y_train):
    '''
    backward pass
    '''
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)
    with tf.GradientTape() as tape:
      tape.watch(self.variables)
      predicted = self.forward(X_train)
      current_loss = self.loss_l1(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients( zip( grads, self.variables))

  def backward_2(self, X_train, y_train):
    '''
    backward pass
    '''
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    with tf.GradientTape() as tape:
      tape.watch(self.variables)
      predicted = self.forward(X_train)
      current_loss = self.loss_l2(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients( zip( grads, self.variables))
    
  def compute_output(self, X):
#    X_tf = tf.cast(X, dytpe=tf.float32)
    what1 = tf.matmul(X, self.W1) + self.b1
    hhat1 = tf.keras.activations.relu(what1)
    what2 = tf.matmul(hhat1, self.W2) + self.b2
    hhat2 = tf.keras.activations.relu(what2)
#    output = tf.nn.softmax(tf.matmul(hhat2, self.Wop) + self.bop)
    output = tf.matmul(hhat2, self.Wop) + self.bop

    return output


In [8]:
NUM_EPOCHS = 10

In [ ]:
mlp_on_gpu = MLP(size_input, size_hidden_1, size_hidden_2, size_output, device='gpu')
m = tf.keras.metrics.SparseCategoricalAccuracy()
time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1],dtype=tf.float32)
  lt = 0
  accuracy = []
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100, seed=epoch*(4208)).batch(100)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total = loss_total + mlp_on_gpu.loss_l1(preds, outputs)
    #lt = lt + mlp_on_gpu.loss(outputs, preds)
    m.update_state(outputs,preds)
    accuracy.append(m.result().numpy())
    mlp_on_gpu.backward_1(inputs, outputs)
  #print('Number of Epoch = {} - Average Loss:= {}'.format(epoch + 1, np.sum(loss_total) / 600))
  m.reset_state()
  print('Number of Epoch = {} - Accuracy:= {} - Average Loss:= {}'.format(epoch + 1, np.sum(accuracy)/len(accuracy), np.sum(loss_total)/X_train.shape[0]))
time_taken = time.time() - time_start
print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Epoch = 1 - Accuracy:= 0.3348852030436198 - Average Loss:= 0.017916996256510418
Number of Epoch = 2 - Accuracy:= 0.6675503540039063 - Average Loss:= 0.011633783976236978
Number of Epoch = 3 - Accuracy:= 0.7160539754231771 - Average Loss:= 0.0095250244140625
Number of Epoch = 4 - Accuracy:= 0.7362216186523437 - Average Loss:= 0.008537568155924479
Number of Epoch = 5 - Accuracy:= 0.749415995279948 - Average Loss:= 0.007952610270182292
Number of Epoch = 6 - Accuracy:= 0.7594038899739584 - Average Loss:= 0.007555587768554688
Number of Epoch = 7 - Accuracy:= 0.7676725260416667 - Average Loss:= 0.007261663818359375
Number of Epoch = 8 - Accuracy:= 0.7745119222005208 - Average Loss:= 0.0070304931640625
Number of Epoch = 9 - Accuracy:= 0.7793938191731771 - Average Loss:= 0.006841766357421875
Number of Epoch = 10 - Accuracy:= 0.7849311319986979 - Average Loss:= 0.0066831466674804685

Total time taken (in seconds): 192.79


In [ ]:
test_loss_total = tf.Variable(0,dtype=tf.float32)
accuracy = []
m = tf.keras.metrics.CategoricalAccuracy()
for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss_l1(preds, outputs)
  m.update_state(outputs,preds)
  accuracy.append(m.result().numpy())
#print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_train.shape[0]))
print('Accuracy: {:.4f} - Loss: {:.4f}'.format(np.sum(accuracy) / len(accuracy), np.sum(test_loss_total.numpy()) / X_train.shape[0]))

In [16]:
seed = 4208
#f = open("/content/drive/My Drive/Colab Notebooks/MLP/FMNIST_adam_noreg.txt","w")
lines = []
time_start = time.time()
for i in range(10):
  print("==============" + str(i)+"=============")
  seed = seed + i

  mlp_on_gpu = MLP(size_input, size_hidden_1, size_hidden_2, size_output, device='cpu')
  m = tf.keras.metrics.SparseCategoricalAccuracy()
  for epoch in range(NUM_EPOCHS):
    loss_total = tf.zeros([1,1],dtype=tf.float32)
    lt = 0
    accuracy = []
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(50, seed=epoch*(seed)).batch(100)
    for inputs, outputs in train_ds:
      preds = mlp_on_gpu.forward(inputs)
      loss_total = loss_total + mlp_on_gpu.loss(preds, outputs)
      lt = lt + mlp_on_gpu.loss(preds, outputs)
      m.update_state(outputs,preds)
      accuracy.append(m.result().numpy())
      mlp_on_gpu.backward(inputs, outputs)
    #print('Number of Epoch = {} - Average Loss:= {}'.format(epoch + 1, np.sum(loss_total) / 600))
    m.reset_state()
    print('Number of Epoch = {} - Accuracy:= {} - Average Loss:= {}\n'.format(epoch + 1, np.sum(accuracy)/len(accuracy), np.sum(loss_total)/X_train.shape[0]))
    lines.append('Number of Epoch = {} - Accuracy:= {} - Average Loss:= {}\n'.format(epoch + 1, np.sum(accuracy)/len(accuracy), np.sum(loss_total)/X_train.shape[0]))
  test_loss_total = tf.Variable(0,dtype=tf.float32)
  accuracy = []
  n = tf.keras.metrics.SparseCategoricalAccuracy()
  for inputs, outputs in test_ds:
    preds = mlp_on_gpu.forward(inputs)
    test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
    n.update_state(outputs,preds)
    accuracy.append(n.result().numpy())
#print('Test MSE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_train.shape[0]))
  lines.append('Test Accuracy: {:.4f} - Loss: {:.4f}\n'.format(np.sum(accuracy) / len(accuracy), np.sum(test_loss_total.numpy()) / X_train.shape[0]))
  print('Test Accuracy: {:.4f} - Loss: {:.4f}\n'.format(np.sum(accuracy) / len(accuracy), np.sum(test_loss_total.numpy()) / X_train.shape[0]))
time_taken = time.time() - time_start
print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

==============0=============
Number of Epoch = 1 - Accuracy:= 0.7053968302408854 - Average Loss:= 0.006436711629231771

Number of Epoch = 2 - Accuracy:= 0.821185048421224 - Average Loss:= 0.005109065246582031

Number of Epoch = 3 - Accuracy:= 0.8362662760416667 - Average Loss:= 0.005213197835286458

Number of Epoch = 4 - Accuracy:= 0.8383838399251302 - Average Loss:= 0.0056189921061197915

Number of Epoch = 5 - Accuracy:= 0.8439528401692709 - Average Loss:= 0.005891758219401042

Number of Epoch = 6 - Accuracy:= 0.8478350830078125 - Average Loss:= 0.0062336090087890626

Number of Epoch = 7 - Accuracy:= 0.8478074137369792 - Average Loss:= 0.006331155904134115

Number of Epoch = 8 - Accuracy:= 0.8534967041015625 - Average Loss:= 0.006327613321940104

Number of Epoch = 9 - Accuracy:= 0.8537467447916667 - Average Loss:= 0.00671036631266276

Number of Epoch = 10 - Accuracy:= 0.8526691691080729 - Average Loss:= 0.007147650146484375

Test Accuracy: 0.8324 - Loss: 0.0086

==============1=======

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/MLP/FMNIST_adam_l1.txt","w") as f:
  for i in lines:
    f.write(i)